<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://cdn.miami.edu/_assets-common/images/system/um-logo-gray-bg.png" alt="Miami Logo" style="height: 98px;">
</div>
    
<div style="float:left; width:120 px; height:120px;">
<img src="http://seasonedchaos.github.io/assets/img/SC_logo.jpg" alt="Seasoned Chaos logo" style="height: 120px;">
</div>

<div style="clear:both"></div>
</div>
    
<h1>The Lorenz "Butterfly Effect" Simulation [Lorenz 1963]</h1><h3><i>Python code by: Kelsey Malloy, Seasoned Chaos team</i></h3>
<br>
<h3>Objective:</h3>Demonstrate chaos theory using Lorenz equations. The three-dimensional chaotic system of equations represents atmospheric convection. The three solutions (seen by the 3 liens) will diverge even if the initial condition differences are barely noticeable by the naked eye. 
<br>
<br>
<br>
<i>last major edit: 07/12/2021


<hr style="height:2px;">

In [1]:
# let's import some libraries
import numpy as np
from scipy.integrate import odeint 
import matplotlib.pyplot as plt
# import matplotlib as mpl
# import mpl_toolkits
import matplotlib.animation as animation
from IPython.display import HTML
import mpl_toolkits.mplot3d.axes3d as p3

### First, here are our functions we will use throughout the notebook.

Lorenz equations:<br>
\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

In [9]:
# defining our Lorenz equations!
def f(y, t, params):
    y1, y2, y3 = y      # unpack current values of y
    r, sigma, b = params  # unpack parameters
    derivs = [sigma*(y2-y1),(r*y1)-y2-(y1*y3),(y1*y2)-(b*y3)]
    return derivs

In [10]:
# define function to draw each frame in 2D
def drawframe2d(n):
    for k in range(0,len(sols)):
        X,Y,Z = sols[k].T
        pts[k].set_data(X[n],Z[n])
        lines[k].set_data(X[:n],Z[:n])
    return pts + lines

In [11]:
# define function to draw each frame in 3D
def drawframe3d(n):
    for k in range(0,len(sols)):
        X,Y,Z = sols[k].T
        pts[k].set_data(X[n],Y[n]) # .set_data doesn't work for 3D plots
        pts[k].set_3d_properties(Z[n])
        lines[k].set_data(X[:n],Y[:n])
        lines[k].set_3d_properties(Z[:n])
        
    return pts + lines

### Now, we must set parameters, which essentially controls the shape of butterfly, and define the initial conditions. We must define the time range over which we want to solve the equations. Finally, we perturb those initial conditions in each dimension, creating 3 solutions/lines for our butterfly to compare.

In [5]:
# set our parameters
sigma = 10.
b = 8./3.
r = 28.
params = [r, sigma, b] # package for ODE solver

# set our initial conditions (ICs)
y1 = -1.
y2 = 3.
y3 = 4.
y = [y1,y2,y3] # package for ODE solver

In [6]:
# Make time array for solution
tStart = 10 # start
tEnd = 30  # end
Nt = 2000 # number of intervals you want
t = np.linspace(tStart,tEnd,Nt)

In [7]:
eps = 10**-1. # perturbation 

# these act like our "ensembles", a fancy word that describes different outcomes based on the slightly different ICs
ens1 = [y[0]+eps,y[1],y[2]]
ens2 = [y[0],y[1]+eps,y[2]]
ens3 = [y[0],y[1],y[2]+eps]
# packaged ensembles
ensset = [ens1,ens2,ens3]

# just some stuff to make our plot pretty
ens_colors = ['royalblue','limegreen','crimson'] # corres. colors
ens_markers = ['o','*','X'] # corres. markers
marker_sizes=[16,14,12] # different sizes just cause it is easier to see all lines/markers in beginning somewhat
line_widths = [4,3.5,3] 

Let's find our solutions!

In [12]:
# get ordinary differential equations (ODE) solutions for ensembles 
sols = [odeint(f,ens,t,args=(params,)) for ens in ensset]

### 2D version of plot to look like the butterfly the most

In [14]:
# Setup figure
fig, ax1 = plt.subplots(figsize=(12,12))
st = fig.suptitle('Lorenz Butterfly & Chaos Theory', fontsize=24)

# Setting the axes properties
ax1.set_xlabel('\nX',fontsize=18)
ax1.set_ylabel('Z\n',fontsize=18)
ax1.set_title('Trajectory in X-Z Phase Space',fontsize=20)
ax1.set_xlim((-20,20))
ax1.set_ylim((0,50))
ax1.tick_params(axis='both', labelsize=12)
#ax1.grid(False)

# create empty points and lines based on # of ensembles
pts = sum([ax1.plot([],[],marker=ens_markers[k],color=ens_colors[k],ms=20) for k in range(0,len(sols))],[])
lines = sum([ax1.plot([],[],color=ens_colors[k],lw=line_widths[k]) for k in range(0,len(sols))],[])

# more attributes for control 
plt.subplots_adjust(top=0.9,bottom=0.1,left=0.1,right=0.9) 
st.set_y(0.95)

# don't show the plain background
plt.close()

# create animiation object and render in HTML video
anim = animation.FuncAnimation(fig, drawframe2d, frames=len(t), interval=15, blit=False)
HTML(anim.to_html5_video())


(3,) (3,)


In [77]:
anim.save('2DLorenzButterfly_XZ.mp4') # save

### 3D version of plot - fun!

In [15]:
# Attaching 3D axis to the figure
fig = plt.figure(figsize=(12,12))
ax1 = p3.Axes3D(fig)
st = fig.suptitle('Lorenz Butterfly & Chaos Theory',fontsize=24)

# NOTE: Can't pass empty arrays into 3d version of plot() so the setup of arrays are different...
pts = [ax1.plot(sol[0, 0:1], sol[1, 0:1], sol[2, 0:1],marker=ens_markers[k],color=ens_colors[k],ms=20)[0] for k,sol in enumerate(sols)]
lines = [ax1.plot(sol[0, 0:1], sol[1, 0:1], sol[2, 0:1],color=ens_colors[k],lw=line_widths[k])[0] for k,sol in enumerate(sols)]

# Setting the axes properties
ax1.set_xlim3d([-25, 25])
ax1.set_xlabel('\nX',fontsize=18)
ax1.set_ylim3d([-25, 25])
ax1.set_ylabel('\nY',fontsize=18)
ax1.set_zlim3d([5, 45])
ax1.set_zlabel('\nZ',fontsize=18)
ax1.tick_params(axis='both', labelsize=12)
#ax1.grid(color='gray',linewidth=0.5)

st.set_y(0.98)

# don't show the plain background
plt.close()

# Creating the Animation object
anim = animation.FuncAnimation(fig, drawframe3d, frames=len(t), interval=15, blit=False)
HTML(anim.to_html5_video())

In [14]:
anim.save('3DLorenzButterfly.mp4') # save